## JuPedSim

In diesem Tutorial wird eine Simulation mit `jpscore` in einem einfachen Setup durchgeführt. 

Eine Dokumentation der verwendeten Dateien ist unter:

- Die Trajektorie-Datei (txt oder xml) (siehe [Dokumentation](http://www.jupedsim.org/jpscore/jpscore_trajectory.html))
- Die Geometriedatei (siehe [Dokumentation](http://www.jupedsim.org/jpscore/jpscore_geometry.html)) 
- Die Projektdatei von jpscore, genannt inifile (siehe [Dokumentation](http://www.jupedsim.org/jpscore/jpscore_inifile.html))


## Wie benutze ich ein Jupyter-Notebook

Dieser `Jupyter-Notebook` besteht aus verschiedennen Zellen (Cells). 

Sie können alle Zellen ausführen indem Sie in dem Menü `Run --> Run All Cells` auswählen.

Alternative, können Sie einzelne Zellen ausführen, indem Sie auf die entsprechende Zelle klicken und `Run --> Run Selected Cells` auswählen. 

**Tip**: Die Ausführung von einer ausgewählten Zelle geht auch mit dem Kürzel `Shift+Enter` (Also auf den Tasten `Alt` und `Enter` gleichzeitig drücken)


## Install 

In [ ]:
%run 00_Install.ipynb
%run config.ipynb

## Load config files

the used inifiles as defined by the user in config.ipynb are loaded

In [ ]:
%store -r jpscore_inifile
%store -r jpsreport_inifile
%store -r jpscore_exe
%store -r jpsreport_exe

jpscore_ini_dir = os.path.dirname(jpscore_inifile)
jpsreport_ini_dir = os.path.dirname(jpsreport_inifile)

In [ ]:
jpscore_exe

## Simulation

Mit den Dateien `ini_jpscore.xml` und `geometry.xml` können wir eine einfache Simulation wie folgt durchführen:

In [ ]:
%%bash -s "$jpscore_exe" "$jpscore_inifile" "$jpscore_ini_dir" 
$1 $2 >log
head -5 log && printf ".\n.\n.\n" && tail -n 4 log
rm -rf $3/results && mv results $3 && mv log $3

In [ ]:
d = parse(jpscore_inifile)
Npeds = int(dict(d.getElementsByTagName('group')[0].attributes.items())['number'])
print("Npeds:", Npeds)

# Visualisierung der Trajektorien

*Warnung*: Die Visualisierung einer großen Trajektoriendatei kann einige Minuten dauern. Also, Geduld! 

Der Name der Geometrie- und Trajektorie-Dateien werden automatisch aus der Ini-Datei extrahiert. 
Darüber hinaus wird auch der Wert der Frames pro Sekunde (*fps*) aus der Ini-Datei extrahiert.

In [ ]:
%%time
%run 02_vis.ipynb
HTML(ani.to_html5_video())

----

## N(t)-Kurve zeichnen

In dem Ordner `results` wird eine Flow-Datei erzeugt, die zwei Spalten enthält:

Abhängig von der Trajektoriennamen könnte diese Datei so heissen `flow_exit_id_0_trajectories.txt`.

**Achtung:**
Wenn die Trajektoriendatei in der `ini_jpscore.xml` geändert wird, muss der Wert der Variable `Nt_file` (siehe Zelle unten) dementsprechend **angepasst** werden.

Zum Beispiel: 

| Trajektorienname  |  Flow-Datei |
| ------------- |:-------------:|
| trajectories50.txt | flow_exit_id_0_trajectories50.txt|
| trajectories100.txt | flow_exit_id_0_trajectories100.txt|
| trajectories150.txt | flow_exit_id_0_trajectories150.txt|



In [ ]:
txt_flow = os.path.join(jpscore_ini_dir, "results", "flow*.txt")
files = glob.glob(txt_flow)
for f in files:
    data = np.loadtxt(f)
    plt.plot(data[:, 1],data[:,0])
    
plt.ylabel("$t$", size=18)
plt.xlabel("$N$", size=18)
plt.grid()

## jpsreport - Methode A

Berechne eine N(t)-Kurve mit der Methode A. Siehe [Documentation](http://www.jupedsim.org/jpsreport_method_A.html).

In [ ]:
d = parse(jpsreport_inifile)
output_dir =  d.getElementsByTagName('output')[0].attributes.items()[0][1]
flow_nt_dir = os.path.join(jpscore_ini_dir, output_dir, "Fundamental_Diagram", "FlowVelocity", "Flow_NT_*trajectories.txt_id_2.dat")
output_dir
flow_nt_dir

In [ ]:
%%bash -s "$jpsreport_exe" "$jpsreport_inifile"
$1 $2

In [ ]:
d = parse(jpsreport_inifile)
files = glob.glob(flow_nt_dir)
for f in files:
    data = np.loadtxt(f)
    plt.plot(data[:, 1], data[:,0])
    
plt.ylabel("$t$", size=18)
plt.xlabel("$N$", size=18)
plt.grid(alpha=0.7)

## Aufräumen

Dieser Schritt ist **optional**. 

Um den Simulationsordners `results` zu löschen, entfernen Sie von der Zelle unten das Zeichen `#` und führen Sie anschliessend die Zelle mit `Alt+Enter` 

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import IntSlider

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

#interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
files = ["Output/Fundamental_Diagram/FlowVelocity/Flow_NT_150trajectories.txt_id_2.dat",
         "Output/Fundamental_Diagram/FlowVelocity/Flow_NT_150trajectories.txt_id_2.dat"]
colors=["red", "green"]
def  slow_function(i):
    f = files[int(i)]
    data = np.loadtxt(f)
    plt.plot(data[:, 1],data[:,0], color=colors[i])
    plt.show()
    
interact(slow_function,i=[0, 1]);



In [ ]:
%pwd


In [ ]:
%pwd

In [ ]:
%%bash
pwd

In [ ]:
%pwd